# Multi Task Model

## Imports

In [54]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import ResNet50

from loadData import createDataset

## Model parameters

In [88]:
# Log parameters
model_name = 'mtm_regression_cascade'
savedModelPath = f'../../log/saved_models/{model_name}'
tb_log_dir = f'../../log/tensorboard/{model_name}'
cp_filepath = f'../../log/cps/{model_name}/'

if not os.path.exists(cp_filepath):
    os.makedirs(cp_filepath)

cp_filepath += 'latest_weights.h5'

# Dynamic hyperparameters
learningRate = 0.001
doDataAugmentation = True
doFineTuning = False
dropoutRate = 0.25
width_multiplier = 1
depth_multiplier = 1

# Training parameters
batch_size = 32
epochs = 2

## Model callbacks

In [56]:
callbacks = [
    # Checkpoint callback                    
    keras.callbacks.ModelCheckpoint(
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=True),

    # Tensorboard callback
    keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1),

    # Early Stopping callback
    # keras.callbacks.EarlyStopping(
    #                 monitor="val_loss",
    #                 patience=2,
    #                 verbose=1)
]

## Data parameters

In [57]:
image_height = 160
image_width = 160

## Data Augmentation

In [58]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomBrightness(0.2),
    ]
)

## Creating the training datasets

In [59]:
x, y_face, y_mask, y_age = createDataset('../../data/m3/training', (image_height, image_width), for_regression=True)

# for x,y in training_ds:
#     for batch in y:
#         print(batch)

## Creating the test dataset

In [60]:
# test_ds = createDataset('../../data/m3/training', batch_size, 0)

## Load the ResNet model

In [61]:
# Loading either the MobileNet architecture model or the previously saved model, and freeze it for transfer learning
mobilenet = ResNet50(
                input_shape=(image_height, image_width, 3), # Optional shape tuple, only to be specified if include_top is False
                # alpha=width_multiplier, # Controls the width of the network. (Width multiplier)
                # depth_multiplier=depth_multiplier, # Depth multiplier for depthwise convolution. (Resolution multiplier)
                # dropout=dropoutRate, # Dropout rate. Default to 0.001.
                weights="imagenet",
                # input_tensor=None,
                # pooling='avg', # Optional pooling mode for feature extraction when include_top is False. (None, avg, max)
                include_top=False
                )
           
# Freeze the base model
mobilenet.trainable = False

inputs = keras.Input(shape=(image_height, image_width, 3))

# Data Augmentation on input
if(doDataAugmentation):
    inputs = data_augmentation(inputs)

# Running base model in inference mode
base_model = mobilenet(inputs, training=False)
base_model = keras.layers.GlobalAveragePooling2D()(base_model)

## Creating Task 1 (Face Detection) Top Model

In [81]:
# Add Dense layer
face_head = tf.keras.layers.Dense(256, activation='relu')(base_model)
face_head = keras.layers.BatchNormalization()(face_head)
face_head = keras.layers.Activation('relu')(face_head)
face_head = keras.layers.Dropout(0.2)(face_head)
face_head = tf.keras.layers.Dense(128, activation='relu')(face_head)
face_head = tf.keras.layers.Dense(64, activation='relu')(face_head)

# Final layer for binary classification
face_outputs = keras.layers.Dense(1, activation='sigmoid')(face_head)

In [82]:
multiply_layer = keras.layers.Multiply()([base_model, face_outputs])
multiply_layer = keras.layers.Dense(512, activation='relu')(multiply_layer)

## Creating Task 2 (Mask Detection) Top Model

In [83]:
# Add Dense layer
mask_head = tf.keras.layers.Dense(128, activation='relu')(multiply_layer)
mask_head = keras.layers.BatchNormalization()(mask_head)
mask_head = keras.layers.Activation('relu')(mask_head)
mask_head = keras.layers.Dropout(0.2)(mask_head)
mask_head = tf.keras.layers.Dense(64, activation='relu')(mask_head)

# Final layer for binary classification
mask_outputs = keras.layers.Dense(1, activation='sigmoid', name='mask_output')(mask_head)

## Creating Task 3 (Age Prediction) Top Model

In [84]:
# Add Dense layer
age_head = tf.keras.layers.Dense(256, activation='relu')(multiply_layer)
age_head = keras.layers.BatchNormalization()(age_head)
age_head = keras.layers.Activation('relu')(age_head)
age_head = keras.layers.Dropout(0.2)(age_head)
age_head = tf.keras.layers.Dense(128, activation='relu')(age_head)
age_head = tf.keras.layers.Dense(64, activation='relu')(age_head)

# Final layer for binary classification
age_outputs = keras.layers.Dense(1, activation='relu', name='age_output')(age_head)

In [85]:
face_outputs = keras.layers.Dense(1, activation='sigmoid', name='face_output')(multiply_layer)

## Creating and compiling the final model

In [86]:

model = keras.Model(inputs, [face_outputs, mask_outputs, age_outputs])
#keras.utils.plot_model(model)

# Using a joint loss function for the three tasks:
# [ Loss = gamma * Loss_task1 + gamma * Loss_task2 + gamma * Loss_task3 ]
# Because every task is dependant on every other task, the model receives the loss of every task when gamma > 0

gamma = 1

model.compile(
      optimizer=keras.optimizers.Adam(), # Learning Rate?
            loss={
                  'face_output': 'mean_squared_error', 
                  'mask_output': 'mean_squared_error',
                  'age_output': 'mean_squared_error'
                  },
            loss_weights={
                  'face_output': 0.33 * gamma,
                  'mask_output': 0.33 * gamma,
                  'age_output': 0.33 * gamma
                  }, 
            metrics={
                  'face_output': 'mean_absolute_error', 
                  'mask_output': 'mean_absolute_error',
                  'age_output': 'mean_absolute_error'
                  },
)

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 5, 5, 2048)   23587712    ['input_9[0][0]']                
                                                                                                  
 global_average_pooling2d_1 (Gl  (None, 2048)        0           ['resnet50[4][0]']               
 obalAveragePooling2D)                                                                            
                                                                                            

## Training the model with the dataset

In [89]:
history = model.fit(
            x,
            y={'face_output': y_face, 
             'mask_output': y_mask,
             'age_output': y_age},
            batch_size=batch_size,
            epochs=epochs, 
            callbacks=callbacks,
            validation_split=0.2,
            shuffle=True
        )

Epoch 1/2
425/425 [==============================] - ETA: 0s - loss: 217.4276 - face_output_loss: 0.0090 - mask_output_loss: 0.0094 - age_output_loss: 658.8534 - face_output_mean_absolute_error: 0.0245 - mask_output_mean_absolute_error: 0.0282 - age_output_mean_absolute_error: 18.0071
Epoch 1: saving model to ../../log/cps/mtm_regression_cascade\latest_weights.h5
425/425 [==============================] - 325s 764ms/step - loss: 217.4276 - face_output_loss: 0.0090 - mask_output_loss: 0.0094 - age_output_loss: 658.8534 - face_output_mean_absolute_error: 0.0245 - mask_output_mean_absolute_error: 0.0282 - age_output_mean_absolute_error: 18.0071 - val_loss: 81.3525 - val_face_output_loss: 0.0015 - val_mask_output_loss: 0.0054 - val_age_output_loss: 246.5157 - val_face_output_mean_absolute_error: 0.0070 - val_mask_output_mean_absolute_error: 0.0108 - val_age_output_mean_absolute_error: 11.9941
Epoch 2/2
425/425 [==============================] - ETA: 0s - loss: 71.2966 - face_output_loss: 0

## Fine tuning

In [ ]:
if doFineTuning:
      model_name = model_name + '_ft'
      mobilenet.trainable = True
      model.summary()

      model.compile(
            optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
            loss={
                  'face_output': keras.losses.BinaryCrossentropy(), 
                  'mask_output': keras.losses.BinaryCrossentropy(),
                  'age_output': keras.losses.SparseCategoricalCrossentropy(ignore_class=-1)
                  },
            loss_weights={
                  'face_output': gamma, 
                  'mask_output': gamma,
                  'age_output': gamma
                  }, 
                  metrics=['accuracy']
      )

      model.fit(training_ds, epochs=epochs)

## Save the model

In [ ]:
# model.save(savedModelPath)

## Test model with test dataset

In [ ]:
# results = model.evaluate(test_ds)

# print(f'Loss: {results[0]}; Accuracy: {results[1]}')

In [ ]:
loaded_model = keras.models.load_model(savedModelPath)

## Predict new images

In [1]:
img_path = '../../data/m3/training/face/mask/1_1_34_900187.jpg'
img_path_split = img_path.split('/')
img_name = img_path_split[len(img_path_split)-1]
img_name_split = img_name.split('_')
face_actual = img_name_split[0]
mask_actual = img_name_split[1]
age_actual = img_name_split[2]

img = tf.keras.utils.load_img(
    img_path, target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array_batch = tf.expand_dims(img_array, 0) # Create a batch

preds = loaded_model.predict(img_array_batch)

face_pred_percent = preds[0][0][0]*100
mask_pred_percent = preds[1][0][0]*100
age_pred = preds[2][0][0]

ax = plt.subplot(1, 1, 1)
plt.imshow(img)
plt.title("Face: {:.2f}% | Mask: {:.2f}% | Age: {:.0f}".format(face_pred_percent, mask_pred_percent, age_pred))

NameError: name 'tf' is not defined